In [34]:
start = "2023-03-11"
end = "2023-04-09"

year = 2024
filename = "form.csv"

In [24]:
import pandas as pd
import json
import datetime
import requests
import os

In [25]:
df = pd.read_csv(filename)
df = df[["Registration Number", "LeetCode"]]
df["Registration Number"] = df["Registration Number"].apply(lambda x: x.lower().strip())
df["Profile"] = df["LeetCode"].apply(lambda x: x.strip().split("https://leetcode.com/")[1].split("/")[0])

df.to_json("form.json", orient="table", index=False)

In [26]:
with open('./form.json', 'r') as f:
    read_data = json.load(f)

advanced_users = read_data['data']

query = """
query userProfileCalendar($username: String!, $year: Int) {  
    matchedUser(username: $username) {
            userCalendar(year: $year) {
                        activeYears
                        streak
                        totalActiveDays  
                        dccBadges {
                            timestamp       
                            badge {
                                name
                                icon
                            }
                        }      
                    submissionCalendar
                    }
                    submitStatsGlobal {
                        totalSubmissionNum {
                            difficulty
                            count
                        }
                        acSubmissionNum {
                            difficulty
                            count
                        }
                        }
                }
            }
"""

headers = {
    "Content-Type": "application/json",
    "Referer": "https://leetcode.com",
}

for user in advanced_users:
    try:
        response = requests.post(
            "https://leetcode.com/graphql",
            headers=headers,
            json={
                "query": query,
                "variables": {"username": user["Profile"], "year": year},
            },
        )
        res = response.json()
        res["registration_number"] = user["Registration Number"]
        res["username"] = user["Profile"]

        with open("./results.json", "a") as f:
            f.write(json.dumps(res) + ',')
    except Exception as e:
        print(str(e))

print({"response": "ok"})

{'response': 'ok'}


In [27]:
# Read the contents of the file
with open('results.json', 'r') as f:
    data = f.read()

# Remove the last character
data = data[:-1]

# Add '[' at the beginning and ']' at the end
data = '[' + data + ']'

# Write the modified contents back to the file
with open('results.json', 'w') as f:
    f.write(data)


In [28]:
# Load the JSON file
with open('results.json') as f:
    data = json.load(f)

In [29]:
results = []
for item in data:
    row = {}

    userCalendar = item["data"]["matchedUser"]["userCalendar"]
    submissionStats = item["data"]["matchedUser"]["submitStatsGlobal"]

    for total in submissionStats["totalSubmissionNum"]:
        row["total" + total["difficulty"]] = total["count"]

    for total in submissionStats["acSubmissionNum"]:
        row["accepted" + total["difficulty"]] = total["count"]

    row.update({ "active_years": userCalendar["activeYears"], "streak": userCalendar["streak"], "total_active_days": userCalendar["totalActiveDays"], "submission_calendar": userCalendar["submissionCalendar"], "username": item["username"], "registration_number": item["registration_number"] })

    results.append(row)

In [30]:
def epoch_to_human(epoch):
    return datetime.datetime.fromtimestamp(epoch).strftime('%Y-%m-%d')

def submissionsInTime(submission_calendar):
    data = json.loads(submission_calendar)
    sum = 0
    for key, value in data.items():
        date = epoch_to_human(int(key))
        if date >= start and date <= end:
            sum += value

    return sum

In [31]:
df = pd.DataFrame(results)
df["submissionsInTime"] = df["submission_calendar"].apply(submissionsInTime)

In [32]:
result = df[["totalAll", "acceptedAll", "submissionsInTime", "username", "registration_number", "streak", "total_active_days", "totalHard", "totalEasy", "acceptedHard", "acceptedEasy"]].sort_values(by="submissionsInTime", ascending=False).reset_index(drop=True)

result.to_csv("result.csv", index=False)

In [33]:
# Specify the file path
file_path = ['results.json', 'form.json']

for file in file_path:
    # Check if the file exists
    if os.path.exists(file):
        # Delete the file
        os.remove(file)
        print(f"The file '{file}' has been deleted.")
    else:
        print(f"The file '{file}' does not exist.")


The file 'results.json' has been deleted.
The file 'form.json' has been deleted.
